## 建立電腦環境變數

In [ ]:
#建立電腦環境變數(模擬的)，並載入模擬環境
from dotenv import load_dotenv
load_dotenv()

## 連線資料庫

In [3]:
import psycopg2
import os
conn = psycopg2.connect(os.environ['postgre_password'])
conn.commit()
conn.close()

In [ ]:
#使用 with as 語法可以自動 commit(), close() conn
with psycopg2.connect(os.environ['postgre_password']) as conn:
    pass